In [1]:
import os
import pandas as pd
import plotly.express as px
import math

def parse_log_file(file_path):
    with open(file_path, 'r') as file:
        numDiracs = None
        numAssets = None
        lines_to_parse = []
        data = []
        line_number = 0

        for line in file:
            line_number += 1
            if line.startswith("numDiracs:"):
                numDiracs = int(line.split(":")[1].strip())
                lines_to_parse = []
            elif line.startswith("numAssets:"):
                numAssets = int(line.split(":")[1].strip())
            elif line.startswith("new - coinsPerByte:") and numDiracs is not None:
                lines_to_parse.append(line)

                if len(lines_to_parse) == numDiracs + 1:
                    first_line = lines_to_parse[1]
                    try:
                        assert all(line == first_line for line in lines_to_parse[1:]), "Not all lines are equal"
                    except AssertionError as e:
                        for line in lines_to_parse:
                            print(line)
                        raise ValueError(f"Assertion error at line {line_number} in file {file_path}: {str(e)}")

                    try:
                        paramBytes, paramLockedAda = [float(x.split(':')[1].strip()) / 1000000 for x in lines_to_parse[0].split('\t')[1:]]
                        diracBytes, diracLockedAda = [float(x.split(':')[1].strip()) / 1000000 for x in lines_to_parse[1].split('\t')[1:]]
                        # paramBytes, paramLockedAda = [math.ceil(float(x.split(':')[1].strip()) / 1000000) for x in lines_to_parse[0].split('\t')[1:]]
                        # diracBytes, diracLockedAda = [math.ceil(float(x.split(':')[1].strip()) / 1000000) for x in lines_to_parse[1].split('\t')[1:]]
                        data.append([file_path, numDiracs, numAssets, paramBytes, paramLockedAda, diracBytes, diracLockedAda])
                    except IndexError as e:
                        raise ValueError(f"Error parsing numbers from lines at line {line_number} in file {file_path}: {str(e)}")
                    lines_to_parse = []

        if data:
            return pd.DataFrame(data, columns=['Filename', 'numDiracs', 'numAssets', 'paramBytes', 'paramLockedAda', 'diracBytes', 'diracLockedAda'])
        else:
            return pd.DataFrame()

def parse_all_logs(directory_path):
    all_dataframes = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):  # Adjust this as per your log files' extension
            full_path = os.path.join(directory_path, filename)
            print(f"Parsing file: {full_path}")
            try:
                df = parse_log_file(full_path)
                all_dataframes.append(df)
            except ValueError as e:
                print(e)  # Print the error message including line number

    return pd.concat(all_dataframes, ignore_index=True)

# Usage:
# df = parse_all_logs('../../test_logs/')
# print(df)


In [2]:
df = parse_all_logs("../../test_logs/");
df;

Parsing file: ../../test_logs/logs_8.txt
Parsing file: ../../test_logs/logs_9.txt
Parsing file: ../../test_logs/logs_18.txt
Parsing file: ../../test_logs/logs_19.txt
Parsing file: ../../test_logs/logs_20.txt
Parsing file: ../../test_logs/logs_11.txt
Parsing file: ../../test_logs/logs_10.txt
Parsing file: ../../test_logs/logs_12.txt
Parsing file: ../../test_logs/logs_13.txt
Parsing file: ../../test_logs/logs_17.txt
Parsing file: ../../test_logs/logs_16.txt
Parsing file: ../../test_logs/logs_14.txt
Parsing file: ../../test_logs/logs_15.txt
Parsing file: ../../test_logs/logs_4.txt
Parsing file: ../../test_logs/logs_5.txt
Parsing file: ../../test_logs/logs_7.txt
Parsing file: ../../test_logs/logs_6.txt
Parsing file: ../../test_logs/logs_2.txt
Parsing file: ../../test_logs/logs_3.txt
Parsing file: ../../test_logs/logs_1.txt


,Filename,numDiracs,numAssets,paramBytes,paramLockedAda,diracBytes,diracLockedAda
0,../../test_logs/logs_8.txt,7,2,0.005773,24.88163,0.000454,1.95674
1,../../test_logs/logs_8.txt,4,2,0.005757,24.81267,0.000439,1.89209
2,../../test_logs/logs_8.txt,5,2,0.005773,24.88163,0.000454,1.95674
3,../../test_logs/logs_8.txt,2,2,0.005767,24.85577,0.000446,1.92226
4,../../test_logs/logs_8.txt,1,2,0.005773,24.88163,0.000446,1.92226
...,...,...,...,...,...,...,...
965,../../test_logs/logs_1.txt,2,2,0.005796,24.98076,0.000470,2.02570
966,../../test_logs/logs_1.txt,1,2,0.005812,25.04972,0.000481,2.07311
967,../../test_logs/logs_1.txt,1,2,0.005801,25.00231,0.000466,2.00846
968,../../test_logs/logs_1.txt,1,5,0.006075,26.18325,0.000659,2.84029


In [3]:
df["extraLockedAda"] = df["paramLockedAda"] - df["diracLockedAda"];
df["extraBytes"] = df["paramBytes"] - df["diracBytes"];
#df["ratio"] = df["diff"] / df["diracLockedAda"];

In [4]:
# Prepare two separate DataFrames for diracBytes and paramBytes
df_dirac = df[['numDiracs', 'numAssets', 'diracBytes', 'diracLockedAda']].copy()
df_dirac['Type'] = 'Dirac'
df_dirac.rename(columns={"diracBytes": "bytes", "diracLockedAda": "lockedAda"}, inplace=True)

df_param = df[['numDiracs', 'numAssets', 'paramBytes', 'paramLockedAda']].copy()
df_param['Type'] = 'Param'
df_param.rename(columns={"paramBytes": "bytes", "paramLockedAda": "lockedAda"}, inplace=True)


df_extra = df[['numDiracs', 'numAssets', 'extraBytes', 'extraLockedAda']].copy()
df_extra['Type'] = 'Extra'
df_extra.rename(columns={"extraBytes": "bytes", "extraLockedAda": "lockedAda"}, inplace=True)

# Concatenate the two DataFrames
df_combined = pd.concat([df_dirac, df_param, df_extra], ignore_index=True)

In [5]:
# Create a scatter plot
fig = px.scatter(
    df_combined, 
    x='numAssets', 
    y='lockedAda',
    # color='numDiracs',
    # size='numDiracs',
    color='Type',  # Different shapes for Dirac and Param
    # color_continuous_scale=px.colors.sequential.Viridis,  # A color scale for dark background
    labels={'numAssets': 'numAssets', 'diracBytes': 'Bytes', 'paramBytes': 'Bytes'},
)

# Update layout for dark background
fig.update_layout(
    plot_bgcolor='rgb(17,17,17)',  # Dark plot background
    paper_bgcolor='rgb(17,17,17)',  # Dark paper background
    font_color='white',  # White text for better contrast
    xaxis=dict(
        gridcolor='rgb(80,80,80)',  # Darker gridlines
    ),
    yaxis=dict(
        gridcolor='rgb(80,80,80)',  # Darker gridlines
    ),
    legend_title_text='Type',
    height=1000
)

# Show the figure
fig.show()


In [6]:
# Create a scatter plot
fig = px.scatter(
    df_combined, 
    x='numDiracs', 
    y='lockedAda',
    color='numAssets',
    symbol='Type',  # Different shapes for Dirac and Param
    # color_continuous_scale=px.colors.sequential.Viridis,  # A color scale for dark background
    labels={'numAssets': 'numAssets', 'diracBytes': 'Bytes', 'paramBytes': 'Bytes'},
)

# Update layout for dark background
fig.update_layout(
    plot_bgcolor='rgb(17,17,17)',  # Dark plot background
    paper_bgcolor='rgb(17,17,17)',  # Dark paper background
    font_color='white',  # White text for better contrast
    xaxis=dict(
        gridcolor='rgb(80,80,80)',  # Darker gridlines
    ),
    yaxis=dict(
        gridcolor='rgb(80,80,80)',  # Darker gridlines
    ),
    legend_title_text='Type',
    height=1000
)

# Show the figure
fig.show()
